# ETL Bronze to Silver
## 1. Importações

In [1]:
import pandas as pd
import numpy as np 
import psycopg2
from psycopg2.extras import execute_batch
import os 
import unicodedata

## 2. Configuração do Banco de Dados


In [18]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'social_media',
    'user': 'postgres',
    'password': 'postgres'
}

INPUT_FILE = '../Data layer/Raw/dados_brutos.csv'
OUTPUT_DIR = 'Data layer/silver/'
OUTPUT_FILE_CSV = 'data_silver.csv'

## 3. Carregamento de dados 

**O processo ETL foi iniciado carregando os dados CSV da pasta Raw com dados brutos**
Foi verificado o tamanho do dataset para enteder o volume de dados que estão sendo tratados 

In [ ]:
print("="*80)
print("ETL BRONZE -> SILVER")
print("="*80)

print("\nCarregando dados Bronze...")
dfR = pd.read_csv(INPUT_FILE)
print(f"   Carregado: {dfR.shape[0]:,} linhas x {dfR.shape[1]} colunas")
print(f"   Memoria: {dfR.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

rows_initial = len(dfR)      

ETL BRONZE -> SILVER

Carregando dados Bronze...
   Carregado: 1,547,896 linhas x 58 colunas
   Memoria: 2101.60 MB


##  4. Removendo colunas inúteis para o objetivo do projeto 

Como nossa regra de negocio é focada em questões de saúde algumas colunas como quantidade de seguidores, se o usuario tem ou não filhos ou se ele tem uma conta premiun não interessa ao estudo e apenas tornam nossa base de dados mais pesada, por isso elas seram removidas.



In [33]:
df_limpo = dfR.drop(columns=['app_name','education_level','has_children','travel_frequency_per_year','posts_created_per_week','dms_sent_per_week','dms_received_per_week','time_on_messages_per_day','time_on_reels_per_day','notification_response_rate','account_creation_year','privacy_setting_level','two_factor_auth_enabled','biometric_login_used','subscription_status','linked_accounts_count','average_session_length_minutes','last_login_date','uses_premium_features','comments_written_per_day',])
print(f"   A nova tabela tem : {df_limpo.shape[0]:,} linhas x {df_limpo.shape[1]} colunas")

   A nova tabela tem : 1,547,896 linhas x 38 colunas


Agora temos uma tabela mais limpa e com colunas que contribuem de alguma forma para o objetivo.

## 5. Especificação maior do países estudados

Decidimos que nossa regra negocio se concentraria em países das americas (Brasil,Canadá e Estados Unidos)

In [39]:
df_limpo = df_limpo[(df_limpo['country'] == 'Brazil') | (df_limpo['country'] == 'Canada') | (df_limpo['country'] == 'United States')]
print(df_limpo['country'].unique())

['Canada' 'United States' 'Brazil']


## Remoção de outliers

Foram detectados outliers em algumas colunas importantes para a construção de nosso dashboard por isso precisamos remove-los 

In [43]:
for col in df_limpo.select_dtypes(include=["float64","int64"]).columns:
    Q1 = df_limpo[col].quantile(0.25)
    Q3 = df_limpo[col].quantile(0.75)
    IQR = Q3 - Q1

    limite_inf = Q1 - 1.5 * IQR
    limite_sup = Q3 + 1.5 * IQR
    
    df_limpo = df_limpo[(df_limpo[col] >= limite_inf) & (df_limpo[col] <= limite_sup)]

print(df_limpo['country'].unique())

['United States' 'Brazil' 'Canada']
